### Importing Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
from pprint import pprint
import numpy as np
import json


### Extract CSV into DataFrames

In [2]:
# Importing Food Information dataset from Kaggle
csv_file = "kaggle_food.csv"
nutrients_df = pd.read_csv(csv_file)
nutrients_df.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,...,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,...,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0.00,193,0,0.00,256,22,...,5,2,0.01,3069,840,0.01,0.001,0.0,2.80,8.6
3,CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,...,256,1395,2.66,763,198,1.22,0.166,0.0,0.25,2.4
4,CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,...,136,560,2.60,1080,292,1.26,0.065,0.0,0.26,2.5


In [3]:
# Importing Recipes by Ingredients dataset from Kaggle
csv_file = "master_recipe.csv"
recipes_df = pd.read_csv(csv_file, index_col=False)
recipes_df.head()

,label,achiote paste,achiote powder,acini di pepe,acorn squash,active dry yeast,adobo sauce,adobo seasoning,adzuki beans,agave nectar,...,yellow rock sugar,yellow squash,yellow summer squash,yellow tomato,yellowfin tuna,yogurt cheese,yucca root,yukon gold potatoes,yuzu juice,zucchini blossoms
0,Infineon Raceway Baked Beans,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sour Cream Noodle Bake,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Middle-Eastern Eggplant Rounds,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Saffron Jewel Rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Thai Sweet and Sour Wings,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Transform Nutrients DataFrame

In [4]:
#Retaining only the columns necessary for the database (Food Information dataset)
lean_nutrient = nutrients_df.drop(columns= [
 'Data.Alpha Carotene',
 'Data.Ash',
 'Data.Beta Carotene',
 'Data.Beta Cryptoxanthin',
 'Data.Choline',
 'Data.Lutein and Zeaxanthin',
 'Data.Lycopene',
 'Data.Manganese',
 'Data.Niacin',
 'Data.Pantothenic Acid',
 'Data.Refuse Percentage',
 'Data.Retinol',
 'Data.Riboflavin',
 'Data.Selenium',
 'Data.Thiamin',
 'Data.Water',
 'Data.Fat.Total Lipid',
 'Data.Household Weights.1st Household Weight',
 'Data.Household Weights.1st Household Weight Description',
 'Data.Household Weights.2nd Household Weight',
 'Data.Household Weights.2nd Household Weight Description',
 'Data.Major Minerals.Calcium',
 'Data.Major Minerals.Copper',
 'Data.Major Minerals.Iron',
 'Data.Major Minerals.Magnesium',
 'Data.Major Minerals.Phosphorus',
 'Data.Major Minerals.Potassium',
 'Data.Major Minerals.Zinc',
 'Data.Vitamins.Vitamin A - IU',
 'Data.Vitamins.Vitamin A - RAE',
 'Data.Vitamins.Vitamin B12',
 'Data.Vitamins.Vitamin B6',
 'Data.Vitamins.Vitamin C',
 'Data.Vitamins.Vitamin E',
 'Data.Vitamins.Vitamin K'])

lean_nutrient.head()

,Category,Description,Nutrient Data Bank Number,Data.Carbohydrate,Data.Cholesterol,Data.Fiber,Data.Kilocalories,Data.Protein,Data.Sugar Total,Data.Fat.Monosaturated Fat,Data.Fat.Polysaturated Fat,Data.Fat.Saturated Fat,Data.Major Minerals.Sodium
0,BUTTER,"BUTTER,WITH SALT",1001,0.06,215,0.0,717,0.85,0.06,21.021,3.043,51.368,576
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0.06,219,0.0,717,0.85,0.06,23.426,3.012,50.489,827
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0.00,256,0.0,876,0.28,0.00,28.732,3.694,61.924,2
3,CHEESE,"CHEESE,BLUE",1004,2.34,75,0.0,353,21.40,0.50,7.778,0.800,18.669,1395
4,CHEESE,"CHEESE,BRICK",1005,2.79,94,0.0,371,23.24,0.51,8.598,0.784,18.764,560


In [5]:
#Renaming the columns in the Food Information (lean_nutrient dataframe)
rename_nutrients = lean_nutrient.rename(columns={"Category" : "ingredient",
                                                "Description" : "food_description",
                                                "Nutrient Data Bank Number" : "ingredient_id",
                                                "Data.Carbohydrate" : "carbs",
                                                 "Data.Protein" : "protein",
                                                "Data.Cholesterol" : "cholesterol",
                                                "Data.Fiber" : "fiber",
                                                "Data.Kilocalories" : "calories",
                                                "Data.Sugar Total" : "sugar",
                                                "Data.Fat.Monosaturated Fat": "monosaturated_fat",
                                                "Data.Fat.Polysaturated Fat" : "polysaturated_fat",
                                                "Data.Fat.Saturated Fat" : "saturated_fat",
                                                "Data.Major Minerals.Sodium" : "sodium"})

rename_nutrients

,ingredient,food_description,ingredient_id,carbs,cholesterol,fiber,calories,protein,sugar,monosaturated_fat,polysaturated_fat,saturated_fat,sodium
0,BUTTER,"BUTTER,WITH SALT",1001,0.06,215,0.0,717,0.85,0.06,21.021,3.043,51.368,576
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0.06,219,0.0,717,0.85,0.06,23.426,3.012,50.489,827
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0.00,256,0.0,876,0.28,0.00,28.732,3.694,61.924,2
3,CHEESE,"CHEESE,BLUE",1004,2.34,75,0.0,353,21.40,0.50,7.778,0.800,18.669,1395
4,CHEESE,"CHEESE,BRICK",1005,2.79,94,0.0,371,23.24,0.51,8.598,0.784,18.764,560
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7408,FROG LEGS,"FROG LEGS,RAW",80200,0.00,50,0.0,73,16.40,0.00,0.053,0.102,0.076,58
7409,MACKEREL,"MACKEREL,SALTED",83110,0.00,95,0.0,305,18.50,0.00,8.320,6.210,7.148,4450
7410,SCALLOP,"SCALLOP,(BAY&SEA),CKD,STMD",90240,0.00,53,0.0,112,23.20,0.00,0.068,0.481,0.146,265
7411,SNAIL,"SNAIL,RAW",90560,2.00,50,0.0,90,16.10,0.00,0.259,0.252,0.361,70


In [6]:
#checking for missing values
null_check = rename_nutrients.isnull().sum().sum()

null_check

0

In [7]:
#saving final dataset as a csv
rename_nutrients.to_csv("output/kaggle_nutrients.csv",index=False)


### Transform Recipes DataFrame

In [8]:
#transposing the Recipe by Ingredients dataframe
transposed_recipes =recipes_df.set_index('label').T

transposed_recipes

label,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,5 Taste Lime Slime Garnished with Mad Rocks,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
achiote paste,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
achiote powder,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acini di pepe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acorn squash,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
active dry yeast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yogurt cheese,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yucca root,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yukon gold potatoes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yuzu juice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#reset the index
transposed_recipes.reset_index(inplace=True)

transposed_recipes

label,index,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,yogurt cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,yucca root,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2921,yukon gold potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2922,yuzu juice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#renaming label column to ingredient
rename_recipe = transposed_recipes.rename(columns={"index" : "ingredient"})

rename_recipe

label,ingredient,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,yogurt cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,yucca root,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2921,yukon gold potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2922,yuzu juice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# changing column names to lower case
master_recipe = rename_recipe.rename(columns=str.lower)
master_recipe

label,ingredient,infineon raceway baked beans,sour cream noodle bake,middle-eastern eggplant rounds,saffron jewel rice,thai sweet and sour wings,balsamic-marinated steak and unstuffed mushrooms,baked chicken saltimbocca,zucchini carpaccio,happy holly's banana cream pie,...,"gianduja caramel ""mud""","strawberry, fig, and goat cheese tarts",eggnog-chocolate cake roll,grilled honey-orange figs with mascarpone and pistachios,pork shoulder rajas with quick kimchi in lettuce leaves with kojuchang sauce,spicy agave wave,apple chimichangas,summer corn salad,zucchini stuffed tomatoes,crabby bisque
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,yogurt cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,yucca root,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2921,yukon gold potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2922,yuzu juice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
recipe_list = master_recipe.columns.str.replace('-', '_').tolist()
master_recipe.columns = recipe_list
recipe_list

['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '5 taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [13]:
#replacing commas with underscores
recipe_list2 = master_recipe.columns.str.replace(',', '_').tolist()
master_recipe.columns = recipe_list2
recipe_list2

['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '5 taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [14]:
#replacing colons with underscores
recipe_list3 = master_recipe.columns.str.replace(':', '_').tolist()
master_recipe.columns = recipe_list3
recipe_list3

['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '5 taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [15]:
#replacing colons with underscores
recipe_list4 = master_recipe.columns.str.replace('(', '_').tolist()
master_recipe.columns = recipe_list4
recipe_list4

<ipython-input-15-be09b365cf44>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  recipe_list4 = master_recipe.columns.str.replace('(', '_').tolist()


['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '5 taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [16]:
recipe_list5 = master_recipe.columns.str.replace(')', '').tolist()
master_recipe.columns = recipe_list5
recipe_list5

<ipython-input-16-90f24c617704>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  recipe_list5 = master_recipe.columns.str.replace(')', '').tolist()


['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '5 taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [18]:
recipe_list6 = master_recipe.columns.str.replace('\d+',"_").tolist()
master_recipe.columns = recipe_list6
recipe_list6

<ipython-input-18-00990aa3c0e9>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_list6 = master_recipe.columns.str.replace('\d+',"_").tolist()


['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 "happy holly's banana cream pie",
 '_ taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 "deano's dragon breath bombs",
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 "grandma lorraine's apple angel food cake",
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pull

In [19]:
recipe_list7 = master_recipe.columns.str.replace("'","").tolist()
master_recipe.columns = recipe_list7
recipe_list7

['ingredient',
 'infineon raceway baked beans',
 'sour cream noodle bake',
 'middle_eastern eggplant rounds',
 'saffron jewel rice',
 'thai sweet and sour wings',
 'balsamic_marinated steak and unstuffed mushrooms',
 'baked chicken saltimbocca',
 'zucchini carpaccio',
 'happy hollys banana cream pie',
 '_ taste lime slime garnished with mad rocks',
 'chicken and quinoa soup',
 'beet salad',
 'baked eggs with green chiles and capers',
 'spinach artichoke dip',
 'tequila bars',
 'strawberry jam',
 'deanos dragon breath bombs',
 'cocoa syrup',
 'pasta with pancetta and lentils',
 'soppressata pizzas',
 'coconut cupcakes with whipped coconut mousse filling and coconut buttercream',
 'muesli parfaits',
 'turkey kofte with apple raita and spinach',
 'soft sugar cookies',
 'cherry and snow pea sprout salad',
 'grandma lorraines apple angel food cake',
 'grilled shrimp salad with sesame ginger vinaigrette',
 'asian rice salad with shrimp',
 'stovetop white cheddar mac',
 'chocoflan',
 'pulled 

In [27]:
recipe_list9 = master_recipe.columns.str.replace(r"[\"]", r"").tolist()
master_recipe.columns = recipe_list9
recipe_list9

<ipython-input-27-bffd2cc06ea0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_list9 = master_recipe.columns.str.replace(r"[\"]", r"").tolist()


['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [28]:
recipe_list10 = master_recipe.columns.str.replace("T\$","").tolist()
master_recipe.columns = recipe_list10
recipe_list10

<ipython-input-28-a6db5d27afc9>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_list10 = master_recipe.columns.str.replace("T\$","").tolist()


['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [36]:
recipe_list11 = master_recipe.columns.str.replace("#","").tolist()
master_recipe.columns = recipe_list11
recipe_list11

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [38]:
recipe_list12 = master_recipe.columns.str.replace(";","").tolist()
master_recipe.columns = recipe_list12
recipe_list12

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [71]:
recipe_list13 = master_recipe.columns.str.replace("&","").tolist()
master_recipe.columns = recipe_list13
recipe_list13

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [72]:
recipe_list14 = master_recipe.columns.str.replace("?","").tolist()
master_recipe.columns = recipe_list14
recipe_list14

<ipython-input-72-d3f631e631ef>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  recipe_list14 = master_recipe.columns.str.replace("?","").tolist()


['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [73]:
recipe_list15 = master_recipe.columns.str.replace("/","").tolist()
master_recipe.columns = recipe_list15
recipe_list15

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [74]:
recipe_list16 = master_recipe.columns.str.replace(".","").tolist()
master_recipe.columns = recipe_list16
recipe_list16

<ipython-input-74-fd31b06c11eb>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  recipe_list16 = master_recipe.columns.str.replace(".","").tolist()


['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [80]:
recipe_list17 = master_recipe.columns.str.replace("*","").tolist()
master_recipe.columns = recipe_list17
recipe_list17

<ipython-input-80-b9c62207dd0c>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  recipe_list17 = master_recipe.columns.str.replace("*","").tolist()


['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [81]:
recipe_list18 = master_recipe.columns.str.replace("=","").tolist()
master_recipe.columns = recipe_list18
recipe_list18

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [82]:
# replacing spaces with underscores in the column names
recipe_list19 = master_recipe.columns.str.replace(' ', '_').tolist()
master_recipe.columns = recipe_list19
recipe_list19

['ingredient',
 'infineon_raceway_baked_beans',
 'sour_cream_noodle_bake',
 'middle_eastern_eggplant_rounds',
 'saffron_jewel_rice',
 'thai_sweet_and_sour_wings',
 'balsamic_marinated_steak_and_unstuffed_mushrooms',
 'baked_chicken_saltimbocca',
 'zucchini_carpaccio',
 'happy_hollys_banana_cream_pie',
 '__taste_lime_slime_garnished_with_mad_rocks',
 'chicken_and_quinoa_soup',
 'beet_salad',
 'baked_eggs_with_green_chiles_and_capers',
 'spinach_artichoke_dip',
 'tequila_bars',
 'strawberry_jam',
 'deanos_dragon_breath_bombs',
 'cocoa_syrup',
 'pasta_with_pancetta_and_lentils',
 'soppressata_pizzas',
 'coconut_cupcakes_with_whipped_coconut_mousse_filling_and_coconut_buttercream',
 'muesli_parfaits',
 'turkey_kofte_with_apple_raita_and_spinach',
 'soft_sugar_cookies',
 'cherry_and_snow_pea_sprout_salad',
 'grandma_lorraines_apple_angel_food_cake',
 'grilled_shrimp_salad_with_sesame_ginger_vinaigrette',
 'asian_rice_salad_with_shrimp',
 'stovetop_white_cheddar_mac',
 'chocoflan',
 'pulled_

In [83]:
#checking for missing values
null_check = master_recipe.isnull().sum().sum()

null_check

0

In [84]:
#final dataset to csv
master_recipe.to_csv("output/kaggle_recipes.csv",index=False)

In [85]:
#preparing the list of columns to import into pgAdmin to create table schema
ingredient_list = pd.DataFrame(recipe_list19) + " " + 'VARCHAR (200)'

ingred_schema= ingredient_list.values.tolist()
ingred_schema

[['ingredient VARCHAR (200)'],
 ['infineon_raceway_baked_beans VARCHAR (200)'],
 ['sour_cream_noodle_bake VARCHAR (200)'],
 ['middle_eastern_eggplant_rounds VARCHAR (200)'],
 ['saffron_jewel_rice VARCHAR (200)'],
 ['thai_sweet_and_sour_wings VARCHAR (200)'],
 ['balsamic_marinated_steak_and_unstuffed_mushrooms VARCHAR (200)'],
 ['baked_chicken_saltimbocca VARCHAR (200)'],
 ['zucchini_carpaccio VARCHAR (200)'],
 ['happy_hollys_banana_cream_pie VARCHAR (200)'],
 ['__taste_lime_slime_garnished_with_mad_rocks VARCHAR (200)'],
 ['chicken_and_quinoa_soup VARCHAR (200)'],
 ['beet_salad VARCHAR (200)'],
 ['baked_eggs_with_green_chiles_and_capers VARCHAR (200)'],
 ['spinach_artichoke_dip VARCHAR (200)'],
 ['tequila_bars VARCHAR (200)'],
 ['strawberry_jam VARCHAR (200)'],
 ['deanos_dragon_breath_bombs VARCHAR (200)'],
 ['cocoa_syrup VARCHAR (200)'],
 ['pasta_with_pancetta_and_lentils VARCHAR (200)'],
 ['soppressata_pizzas VARCHAR (200)'],
 ['coconut_cupcakes_with_whipped_coconut_mousse_filling_an

In [25]:
#loading dependency to encode text file with UTF-8 encoding
import io

In [86]:
#writing the schema to a text file 
with open ('ingredient_list.txt', 'w', encoding='utf8') as my_ingred_list:
    for x in ingred_schema:
        my_ingred_list.write("%s\n" % x)
    print('List written.')

List written.


### Load DataFrames into database

In [ ]:
#checking data types
rename_nutrients.dtypes


In [ ]:
#import dependencies
from sqlalchemy import create_engine
import psycopg2


In [ ]:
#setting up the connection to pgAdmin
rds_connection_string = "postgres:postgres@localhost:5432/kaggle_nutrition_db"

engine = create_engine(f'postgresql://{rds_connection_string}')


In [ ]:
#obtaining table names
engine.table_names()

In [ ]:
#reading in the kaggle_nutrients.csv file into SQL
df = pd.read_csv ('output/kaggle_nutrients.csv')

df.head()


In [ ]:
#adding dataframe to SQL
df.to_sql(name='kaggle_nutrition', con=engine, if_exists='append', index=False)


In [ ]:
#reading in the kaggle_recipes.csv file into SQL
df2 = pd.read_csv ('output/kaggle_recipes.csv')

df2.head()

In [ ]:
#adding dataframe to SQL
df.to_sql(name='kaggle_recipes', con=engine, if_exists='append', index=False)
